In [1]:
!pip install super-json-mode openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.2/65.2 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.1/226.1 kB 38.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.9/381.9 kB 45.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 133.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 117.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from superjsonmode.integrations.openai import StructuredOpenAIModel
from pydantic import BaseModel


# Remember to set your OPENAI_API_KEY env variable
# or pass in `api_key` to the constructor below
model = StructuredOpenAIModel()

In [3]:
class QuarterlyReport(BaseModel):
    company: str
    stock_ticker: str
    date: str
    reported_revenue: str
    dividend: str

In [4]:
prompt_template = """{prompt}

Based on this excerpt, extract the correct value for the provided key. Keep it succinct. It should be a {type}.

{key}: """

prompt = """NVIDIA Announces Financial Results for Third Quarter Fiscal 2024
November 21, 2023
Record revenue of $18.12 billion, up 34% from Q2, up 206% from year ago
Record Data Center revenue of $14.51 billion, up 41% from Q2, up 279% from year ago
NVIDIA (NASDAQ: NVDA) today reported revenue for the third quarter ended October 29, 2023, of $18.12 billion, up 206% from a year ago and up 34% from the previous quarter.

GAAP earnings per diluted share for the quarter were $3.71, up more than 12x from a year ago and up 50% from the previous quarter. Non-GAAP earnings per diluted share were $4.02, up nearly 6x from a year ago and up 49% from the previous quarter.

“Our strong growth reflects the broad industry platform transition from general-purpose to accelerated computing and generative AI,” said Jensen Huang, founder and CEO of NVIDIA.

“Large language model startups, consumer internet companies and global cloud service providers were the first movers, and the next waves are starting to build. Nations and regional CSPs are investing in AI clouds to serve local demand, enterprise software companies are adding AI copilots and assistants to their platforms, and enterprises are creating custom AI to automate the world’s largest industries.

“NVIDIA GPUs, CPUs, networking, AI foundry services and NVIDIA AI Enterprise software are all growth engines in full throttle. The era of generative AI is taking off,” he said.

NVIDIA will pay its next quarterly cash dividend of $0.04 per share on December 28, 2023, to all shareholders of record on December 6, 2023."""


In [5]:
import time

start = time.time()
output = model.generate(
    prompt,
    extraction_prompt_template=prompt_template,
    schema=QuarterlyReport,
    batch_size=6,
    temperature=0,
)
print(f"Total time: {time.time() - start}")

Total time: 0.37114882469177246


In [6]:
output

{'company': 'NVIDIA',
 'stock_ticker': 'NVDA',
 'date': 'November 21, 2023',
 'reported_revenue': '$18.12 billion',
 'dividend': '0.04'}

In [9]:
start = time.time()
output = model.default_generate(
    prompt,
    schema=QuarterlyReport,
    temperature=0,
)
print(f"Total time: {time.time() - start}")

Total time: 2.8135507106781006


### Let's compare it to naive generation using the Completions endpoint...

In [10]:
output

'\n\nCompany: NVIDIA\nQuarter: Third Quarter Fiscal 2024\nRevenue: $18.12 billion\nPercentage Change from Previous Quarter: 34%\nPercentage Change from Previous Year: 206%\nData Center Revenue: $14.51 billion\nPercentage Change from Previous Quarter: 41%\nPercentage Change from Previous Year: 279%\nGAAP Earnings per Diluted Share: $3.71\nPercentage Change from Previous Quarter: 50%\nPercentage Change from Previous Year: 12x\nNon-GAAP Earnings per Diluted Share: $4.02\nPercentage Change from Previous Quarter: 49%\nPercentage Change from Previous Year: 6x\nDividend: $0.04 per share\nPayment Date: December 28, 2023\nRecord Date: December 6, 2023'